**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

In [6]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## <font color='blue'> Q4 [25 points total - answer all parts] </font>

## <font color='blue'> To protect your e-banking passwords, a friend has developed a password manager that uses an encrypted database to store your credentials for each bank account. To unlock the password manager and access your credentials you must know the correct 16-byte passphrase. The password manager program keeps a local .txt file containing the hex-encoded encrypted copy of the correct passphrase using AES-128 encryption (i.e., encrypted_passphrase = convert_to_hex(AES-128(aes_key, plaintext_passphrase)). Moreover, the password manager uses AES-128 in CBC mode with the correct “aes_key” to decrypt the ciphertexts of the e-banking credentials stored in the password database. Notably, since these credentials can have an arbitrary length in bytes, the CBC decryption checks if the padding after decryption is correct. </font>

## <font color='blue'> Unfortunately, you forgot your passphrase for the password manager so you can no longer access your e-banking credentials in the encrypted database. Your only hope is to decrypt the local copy of the encrypted passphrase, but you don’t know what is the random AES key used (i.e., you don’t know “aes_key” above). Even though the password manager has a function called “call_olivia(ctxt, iv)” that decrypts AES-CBC ciphertexts and checks if the padding is correct, it only returns True (i.e., no error) or False (there was an error).</font>

## <font color='blue'> You are given black-box access to the call_olivia(ctxt, iv) function binary and the hex-encoded AES-128 ciphertext of your passphrase. </font>

### <font color='blue'> Q4-a: [5 points] Explain what steps you need to follow in order to recover your passphrase. What exactly needs to be done in each step?  </font>

The ciphertext is split into 16-byte blocks , I'll modify the last block of the ciphertext and check if the padding is valid using the call_olivia function. I will iterate through all the possible byte values till the padding is valid using which I can find the last byte of the decrypted ciphertext.I will repeat this for the entire block. After decrypting the entire block, I will strip the padding and recover the passphrase.


### <font color='blue'> Q4-b: [20 points] Correctly Implement function get_passphrase(ctxt) below that can recover the passphrase plaintext by invoking call_olivia(). The hex-encoded password is ctxt = "a70adfcef4e67f1bdfcabbb8d1878947". What is the passphrase returned by your program?</font>

In [1]:
import socket
import time

def olivia_init():
    global s
    server_address = '3.142.243.61'
    server_port = 12345

    # Create the socket
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Setup the connection
    try:
        # Setup the connection
        s.connect((server_address, server_port))
        print("Connection established.")
    except Exception as e:
        print(f"Error establishing connection: {e}")
        s = None

def olivia_close():
    global s
    if s:
        s.close()
        print("connection closed.")
        s = None

def call_olivia(ctxt: str, iv: str):
    global s
    
    if s is None:
        raise ConnectionError("Socket is not initialized. Call olivia_init first.")

    message = f'--iv {iv} --ctxt {ctxt}\n'
    
    # Send the message
    try:
        s.sendall(message.encode())
    except Exception as e:
        print("no message", e)
    
    try:
        response = s.recv(1024)
    except Exception as e:
        print("no recv", e)
        return False

    
    # Decode the response
    tmp = response.decode()
    if "False" in tmp:
        return False
    return True

    

def xor(x,y):
    if x == "" or y == "":
        return ""
    x = int(x,16)
    y = int(y,16)
    z = x ^ y
    return hex(z)[2:].zfill(2)


In [2]:
# 5 points
def create_iv_tail(ptxt: str, i: int) -> str:
    """
    Input is ptxt in hex string format, and i is the number of padding bytes needed.
    create the iv_tail that will lead to the correct padding generation in the ptxt.
    """

    padding = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7: '07', 8: '08', 9: '09', 10: '0a', 11: '0b', 12: '0c', 13: '0d', 14: '0e', 15: '0f', 16: '10', 17: '011'}
    
    iv_tail = ""

    # YOUR CODE HERE
    if not ptxt:
        return iv_tail
    ptxt_bytes=bytes.fromhex(ptxt) # convert otxt to bytes
    padding_value=int(padding[i],16) # convert value to int
    for j in ptxt_bytes:
        padded_byte=j^padding_value # xor ptxt and padding_value
        iv_tail += hex(padded_byte)[2:].zfill(2) # add padded_byte to iv_tail
    #raise NotImplementedError()

    return iv_tail

In [3]:
assert create_iv_tail("" , 1) == ""
assert create_iv_tail("76" , 2) == "74"
assert create_iv_tail("1234" , 3) == "1137"
assert create_iv_tail("12334adf32fd" , 5) == "17364fda37f8"


In [4]:
# hidden test cases

In [7]:
# worth 5 points
def find_correct_byte(ctxt: str , iv_tail: str) -> str:
    """
    Given the ctxt and the iv_tail from the previous step, test each possible iv using the iv_tail to find the correct_byte.
    You should return the byte in hex format. Use the call_olivia(ctxt, iv) function, and find the iv that matches the ctxt.
    iv should be hex string of size 32. The correct_byte should be returned in hex string format.
    
    """
        

    correct_byte = ""
    # YOUR CODE HERE
    for i in range(256):
        current_byte=format(i,'02x') 
        zero_padding_length = (16 - len(iv_tail) // 2 - 1)  # length of the iv_tail+current_byte 
        zero_padding = '00' * max(zero_padding_length, 0)  # length of iv
        iv = zero_padding  + current_byte + iv_tail # initialise iv
        if len(iv) != 32:
            continue
        if call_olivia(ctxt, iv):
            return current_byte  
    return None 
    #raise NotImplementedError()

    return correct_byte


In [8]:
olivia_init()

ctxt = "a70adfcef4e67f1bdfcabbb8d1878947"
correct_byte = find_correct_byte(ctxt,"")
assert correct_byte == "76"

ctxt = "a70adfcef4e67f1bdfcabbb8d1878947"
correct_byte = find_correct_byte(ctxt,"75")
assert correct_byte == "72"

ctxt = "a70adfcef4e67f1bdfcabbb8d1878947"
correct_byte = find_correct_byte(ctxt,"7374")
assert correct_byte == "5c"

olivia_close()

Connection established.
connection closed.


In [9]:
# hidden test cases

In [21]:
# worth 10 points
def get_passphrase(ctxt: str) -> str:
    """
    Your function should use the call_olivia function to do a padding oracle attack on the ctxt = a70adfcef4e67f1bdfcabbb8d1878947
    and return it's plaintext in string format.
    At each padding step, use the create_iv_tail function to create the end of the iv, and then call the find_correct_byte function
    to bruteforce the correct byte, then use the byte to find it's plaintext. Add all the bytes in plaintext then decode and 
    return it.
    """

    
    padding = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7: '07', 8: '08', 9: '09', 10: '0a', 11: '0b', 12: '0c', 13: '0d', 14: '0e', 15: '0f', 16: '10'}
    ptxt = ""
    # YOUR CODE HERE
    #raise NotImplementedError()
    iv_tail = ""
    
    
    for i in range(1, 17):
        iv_tail = create_iv_tail(ptxt, i) # create iv tail
        correct_byte = find_correct_byte(ctxt, iv_tail) # find valid byte
        ptxt += correct_byte #form ptxt
    passphrase = bytes.fromhex(ptxt).decode('iso-8859-1')
    return passphrase


In [22]:
olivia_init()
ctxt = "a70adfcef4e67f1bdfcabbb8d1878947"
ptxt = get_passphrase(ctxt)
print("ptxt is: ", ptxt)
olivia_close()
with open("pass_phrase.txt","w") as file:
    file.write(ptxt)


Connection established.


TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
# hidden test